In [ ]:
'''
    for DataSet DA-20k
    LHABS:   'tbpcqi-labs-0604-122216'
    LBS:    'dgfzzb-lbs-0604-124454'
    LHAB:    'ynukky-lab-0604-133426'
    Basic:  'gkrqzx-b-0608-122552'

'''

In [ ]:
# Load the model and get the attention layer
'''
LHABS	logs/DA-20k/cut/LHABS/tbpcqi-labs-0604-122216/model/checkpoint_labs.h5
LHAB    logs/DA-20k/cut/LHAB/iwprfa-lab-0604-133738/model/checkpoint_lab.h5
LBS    logs/DA-20k/cut/LBS/dgfzzb-lbs-0604-124454/model/checkpoint_lbs.h5
'''
%cd MathByte
from models.evaluation_metrics import precision_1k,precision_2k, precision_3k, precision_5k, recall_1k, recall_2k,recall_3k, recall_5k, F1_1k,F1_2k, F1_3k, F1_5k, lcm_metrics
import models.lstm
import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Flatten, Dropout, Concatenate, Lambda, Multiply, Reshape, Dot, Bidirectional
import keras.backend as K
import utils


logdir = "logs/DA-20k/cut/LBS/dgfzzb-lbs-0604-124454/model/checkpoint_lbs.h5"
use_att = "_labs" in logdir or "_lab" in logdir
use_lcm = "_labs" in logdir or "_lbs" in logdir
if use_att:
    layer_name_0 = "0_attention_layer_att_weight"
    layer_name_1 = "1_attention_layer_att_weight" # att_context
loss, metrics = lcm_metrics(427, 4)

saved_model = load_model(logdir, custom_objects={
    "K": K,
    "precision_1k": precision_1k,
    "precision_2k": precision_2k,
    "precision_3k": precision_3k,
    "precision_5k": precision_5k,
    "recall_1k": recall_1k,
    "recall_2k": recall_2k,
    "recall_3k": recall_3k,
    "recall_5k": recall_5k,
    "F1_1k": F1_1k,
    "F1_2k": F1_2k,
    "F1_3k": F1_3k,
    "F1_5k": F1_5k,
    "lcm_loss": loss,
    "lcm_precision_1k": metrics[0],
    "lcm_precision_2k": metrics[1],
    "lcm_precision_3k": metrics[2],
    "lcm_precision_5k": metrics[3],
    "lcm_recall_1k": metrics[4],
    "lcm_recall_2k": metrics[5],
    "lcm_recall_3k": metrics[6],
    "lcm_recall_5k": metrics[7],
    "lcm_f1_1k": metrics[8],
    "lcm_f1_2k": metrics[9],
    "lcm_f1_3k": metrics[10],
    "lcm_f1_5k": metrics[11],
    "lcm_accuracy_1k": metrics[12],
    "lcm_accuracy_2k": metrics[13],
    "lcm_accuracy_3k": metrics[14],
    "lcm_accuracy_5k": metrics[15],
    "lcm_hamming_loss_k": metrics[16]
},compile = False)
# saved_model.summary()
print(saved_model.summary())
if use_att:
    model = Model(inputs=saved_model.input,outputs=[saved_model.output, saved_model.get_layer(layer_name_0).output, saved_model.get_layer(layer_name_1).output])
else:
    model = Model(inputs=saved_model.input,outputs=[[saved_model.output]])
print(use_lcm)

In [2]:
# read the data
import h5py
import numpy as np
f_data = h5py.File('../file_data/DA-20k/math_data_cut.h5', 'r')
import pickle
cache_file_pickle = "../file_data/DA-20k/vocab_label.pkl"
word2index, label2index = None, None
with open(cache_file_pickle, 'rb') as data_f_pickle:
    word2index, label2index = pickle.load(data_f_pickle)
index2word = {}
index2label = {}
for key in word2index:
  index2word[word2index[key]] = key
for key in label2index:
  index2label[label2index[key]] = key
print(index2word[0], len(label2index), len(index2word))

<pad> 427 82595


In [3]:
# use the TestSet to test the model
test_X = f_data['X'][()][16893:17000] # id 50779 index 5457  # id 45936 index 641  # id 62271 index 16893
test_Y = f_data['y'][()][16893:17000]
L_test = np.array([np.array(range(442))
                          for i in range(len(test_X))])
print(len(test_X))
print(len(test_Y))
print(test_X)

107
107
[[  59    3  965 ...    0    0    0]
 [  96    3   32 ...    0    0    0]
 [ 105    3   32 ...    0    0    0]
 ...
 [ 126    3   78 ...    0    0    0]
 [ 130    3    4 ...    0    0    0]
 [ 133    3 4341 ...    0    0    0]]


In [4]:
# read the weights of the model
outputs = model.predict([test_X, L_test],batch_size = 512)
model_outputs = outputs[0]
if use_att:
  attention_outputs_0 = outputs[1]
  attention_outputs_1 = outputs[2]
if use_lcm:
  # lcm model
  model_outputs = model_outputs[:, :427]
def sigmoid(z):
  return 1/(1 + np.exp(-z))


In [ ]:
import numpy as np

which_one = 0
# before the prediction i supposed you tokenized text
tokenized_text = [index2word[item] for item in test_X[which_one]]
# if you are using batches the outputs will be in batches
# get exact attentions of chars
level_1_attention = []
labels = []
for i, val in enumerate(test_Y[which_one]):
  if val==1:
    labels.append(index2label[i])
    if use_att:
      level_1_attention.append(attention_outputs_1[which_one][i])

pred_labels = []
rank_mat = np.argsort(model_outputs[which_one])
backup = np.copy(model_outputs[which_one])
# for m in range(len(labels)):
for m in range(3):
  y_pred = np.copy(backup)
  y_pred[rank_mat[:-(m + 1)]] = 0
  y_pred = np.ceil(y_pred)
for i, val in enumerate(y_pred):
  if val==1:
    pred_labels.append(index2label[i])
print(','.join(tokenized_text))
print("True knowledge concepts:", labels)
print("Pred knowledge concepts:", pred_labels)

level_1_attention_output = np.mean(np.array(level_1_attention),axis=0).tolist()

In [ ]:
# 5.2 get the distribution of label-semantic attention
class CharVal(object):
    def __init__(self, char, val):
        self.char = char
        self.val = val

    def __str__(self):
        return self.char

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
def color_charvals(s):
    r = 255-int(s.val*255) # 100
    if r > 120:
      r = 255
    color = rgb_to_hex((255, r, r))
    return 'background-color: %s' % color

if use_att:
    # you need to match each char and attention
    char_vals = [CharVal(c, v) for c, v in zip(tokenized_text, level_1_attention_output)]
    import pandas as pd
    char_df = pd.DataFrame(char_vals).transpose()
    # apply coloring values
    char_df = char_df.style.applymap(color_charvals)
char_df


In [ ]:
# 5.3 Analysis of multi-label smoothing combining textual features
if use_lcm:
  outs = outputs[0] # output
  label_sim_dist = outs[:, 427:] # lcm simulated labels
  pred_probs = outs[:, :427] # lcm predicted labels
  simulated_y_true = K.softmax(label_sim_dist+4*test_Y)

  which_simulated_y = simulated_y_true[which_one] # simulated labels
  ground_y = test_Y[which_one] # ground truth
  pred = K.softmax(pred_probs[which_one]) # predicted labels

  import matplotlib.pyplot as plt
  x_data = [i for i in range(427)]
  # draw the simulated label distribution
  plt.plot(x_data,which_simulated_y,color='blue')
  for i in range(len(x_data)):
    if(ground_y[i]==1):
      plt.vlines(i, 0, 0.2, colors = "orange", linestyles = "dashed")
  plt.show()
  # draw the predicted label distribution
  plt.plot(x_data,pred_probs[which_one],color='blue')
  for i in range(len(x_data)):
    if(ground_y[i]==1):
      plt.vlines(i, 0, 1, colors = "orange", linestyles = "dashed")
  plt.show()